In [32]:
wsj0path = "/project/data_asr/wham_dataset/whamr_data/wsj0_raw/"

cv = []
with open('mix_2_spk_cv.txt','r') as txt:
    out = txt.readline()
    while out != "":
        out = out.split()
        cv.append([out[0],out[2]])
        out = txt.readline()

tr = []
with open('mix_2_spk_tr.txt','r') as txt:
    out = txt.readline()
    while out != "":
        out = out.split()
        tr.append([out[0],out[2]])
        out = txt.readline()

tt = []
with open('mix_2_spk_tt.txt','r') as txt:
    out = txt.readline()
    while out != "":
        out = out.split()
        tt.append([out[0],out[2]])
        out = txt.readline()

print("tr:",len(tr))
print("cv:",len(cv))
print("tt:",len(tt))
print(tr[0])

tr: 20000
cv: 5000
tt: 3000
['wsj0/si_tr_s/40n/40na010x.wav', 'wsj0/si_tr_s/01x/01xo031a.wav']


In [33]:
import torchaudio

a,b = torchaudio.load(wsj0path+tr[0][0])
c,d = torchaudio.load(wsj0path+tr[0][1])
print(a)
print(c)

tensor([[-3.0518e-05, -1.2207e-04,  1.2207e-04,  ..., -3.0518e-05,
         -9.1553e-05, -2.4414e-04]])
tensor([[ 6.1035e-05,  1.8311e-04,  6.1035e-05,  ...,  3.3569e-04,
         -3.0518e-05,  9.1553e-05]])
